In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

import time

from model import Generator, Discriminator
from config import irgan_config
from data_utils import RecDataset, DataProvider
from evaluation.rec_evaluator import RecEvaluator

In [2]:
epochs = irgan_config.epochs
batch_size = irgan_config.batch_size
epochs_d = 15
epochs_g = 50
emb_dim = irgan_config.emb_dim
eta_G = irgan_config.eta_G
eta_D = irgan_config.eta_D
device = irgan_config.device
weight_decay_g = irgan_config.weight_decay_g
weight_decay_d = irgan_config.weight_decay_d

In [3]:
# Hyper-parameters and datset-specific parameters
rec_dataset = RecDataset(irgan_config.dir_path)
all_users = rec_dataset.get_users()
all_items = rec_dataset.get_items()
num_users = rec_dataset.get_num_users()
num_items = rec_dataset.get_num_items()
bought_mask = rec_dataset.get_bought_mask().to(device)
eval_dict = rec_dataset.get_interaction_records("test")
train_ui = rec_dataset.get_user_item_pairs()

In [4]:
dp = DataProvider(device)
evaluator = RecEvaluator(eval_dict, None, device)

In [5]:
writer_dis = SummaryWriter("runs/IRGAN-Adam-dns-gen/Discriminator")
writer_gen = SummaryWriter("runs/IRGAN-Adam-dns-gen/Generator")

In [6]:
G = Generator(num_users, num_items, emb_dim, bought_mask)
D = Discriminator(num_users, num_items, emb_dim)
G = G.to(device)
D = D.to(device)

loss_D = nn.BCELoss()
optimizer_G = torch.optim.Adam(G.parameters(), lr = eta_G)
optimizer_D = torch.optim.Adam(D.parameters(), lr = eta_D, weight_decay = weight_decay_d)

In [7]:
pretrained_model = torch.load("./pretrained_models/pretrained_model_dns.pkl",map_location=irgan_config.device)
G.load_state_dict(pretrained_model)

<All keys matched successfully>

In [8]:
for epoch in range(epochs):
    print(
            "[Epoch %d/%d]"
            % (epoch+1, epochs)
        )  
    time_epoch_start = time.time()
    
    # ---------------------
    #  Train Discriminator
    # ---------------------   
    D.train()
    G.eval()
    for epoch_d in range(epochs_d):
        time_d_start = time.time()
        if(epoch_d % 5 == 0):
            train_set_d = dp.prepare_data_for_discriminator(G, train_ui, batch_size = batch_size)
        loss_d_epoch = 0
        for users, items, labels in train_set_d:
            dis_score = D(users,items)
            loss_d = loss_D(dis_score, labels)
            loss_d_epoch += loss_d.item()
            optimizer_D.zero_grad()
            loss_d.backward()
            optimizer_D.step()
            
        time_d_end = time.time()
        loss_d_epoch /= len(train_set_d)
        print(
            "\t[Discriminator][Epochs %d/%d] [D epoch loss: %6.5f] [Time:%6.5f] "
            % (epoch_d+1, epochs_d, loss_d_epoch, time_d_end - time_d_start)
        )
        writer_dis.add_scalar('Loss/Discriminator', loss_d_epoch, epoch*epochs_d + epoch_d)
        with torch.no_grad():
            res = evaluator.top_k_evaluation(D, [3,5,10])    
            ndcg3, precision3, hit3, map3, mrr3 = res[0]
            ndcg5, precision5, hit5, map5, mrr5 = res[1]
            ndcg10, precision10, hit10, map10, mrr10 = res[2]
            writer_dis.add_scalar('Metrics/NDCG@3', ndcg3, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/NDCG@5', ndcg5, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/NDCG@10', ndcg10, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/Precision@3', precision3, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/Precision@5', precision5, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/Precision@10', precision10, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/Hit@3', hit3, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/Hit@5', hit5, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/Hit@10', hit10, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/MAP', map10, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/MRR', mrr10, epoch*epochs_d + epoch_d)
        
    # --------------------- 
    #  Train Generator
    #
    # For generator
    #   Generate K user-item pairs
    #   Leveraging Policy Gradient to update parameters of generator
    #
    # --------------------- 
    D.eval()
    G.train()
    for epoch_g in range(epochs_g):
        time_g_start = time.time()
        train_set_g = dp.prepare_data_for_generator(all_users, batch_size)
        loss_g_epoch = 0
        for fake_users, in train_set_g:
            fake_items, fake_probs, fake_p_n = G.sample_items_for_users(fake_users, k = 256,\
                                                                        temperature=1, lambda_bought=0.2)
            fake_users = fake_users.view(-1,1).expand_as(fake_items).contiguous()
            fake_users = fake_users.view(-1)
            fake_items = fake_items.view(-1)
            fake_probs = fake_probs.view(-1)
            fake_p_n = fake_p_n.view(-1)
            
            log_fake_probs = torch.log(fake_probs.clamp(1e-8))
            fake_probs = fake_probs.detach()
            fake_p_n = fake_p_n.detach()
            
            reward = (2 * D(fake_users, fake_items) - 1).detach()*(fake_probs/fake_p_n)
            loss_g = -torch.mean(log_fake_probs*reward)
            loss_g_epoch += loss_g
            
            optimizer_G.zero_grad()  
            loss_g.backward() 
            optimizer_G.step()
        time_g_end = time.time()
        print(
            "\t[Generator][Epochs %d/%d] [G epoch loss: %6.5f] [Time:%6.5f]"
            % (epoch_g + 1, epochs_g, loss_g_epoch, time_g_end - time_g_start )
        ) 
        writer_gen.add_scalar('Loss/Generator', loss_g_epoch, epoch*epochs_g + epoch_g)
        with torch.no_grad():
            res = evaluator.top_k_evaluation(G, [3,5,10])    
            ndcg3, precision3, hit3, map3, mrr3 = res[0]
            ndcg5, precision5, hit5, map5, mrr5 = res[1]
            ndcg10, precision10, hit10, map10, mrr10 = res[2]
            writer_gen.add_scalar('Metrics/NDCG@3', ndcg3, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/NDCG@5', ndcg5, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/NDCG@10', ndcg10, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/Precision@3', precision3, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/Precision@5', precision5, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/Precision@10', precision10, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/Hit@3', hit3, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/Hit@5', hit5, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/Hit@10', hit10, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/MAP', map10, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/MRR', mrr10, epoch*epochs_g + epoch_g)
    time_epoch_end = time.time()
writer_dis.close()    
writer_gen.close()

[Epoch 1/200]
	[Discriminator][Epochs 1/15] [D epoch loss: 0.60916] [Time:6.40928] 
	[Discriminator][Epochs 2/15] [D epoch loss: 0.38940] [Time:8.60304] 
	[Discriminator][Epochs 3/15] [D epoch loss: 0.27121] [Time:9.33740] 
	[Discriminator][Epochs 4/15] [D epoch loss: 0.22011] [Time:9.45300] 
	[Discriminator][Epochs 5/15] [D epoch loss: 0.19828] [Time:9.31546] 
	[Discriminator][Epochs 6/15] [D epoch loss: 0.18573] [Time:8.94045] 
	[Discriminator][Epochs 7/15] [D epoch loss: 0.17614] [Time:9.31754] 
	[Discriminator][Epochs 8/15] [D epoch loss: 0.16795] [Time:9.33055] 
	[Discriminator][Epochs 9/15] [D epoch loss: 0.16082] [Time:9.98912] 
	[Discriminator][Epochs 10/15] [D epoch loss: 0.15464] [Time:10.96248] 
	[Discriminator][Epochs 11/15] [D epoch loss: 0.14936] [Time:11.41541] 
	[Discriminator][Epochs 12/15] [D epoch loss: 0.14474] [Time:11.43609] 
	[Discriminator][Epochs 13/15] [D epoch loss: 0.14073] [Time:11.03147] 
	[Discriminator][Epochs 14/15] [D epoch loss: 0.13721] [Time:9.90317

	[Generator][Epochs 43/50] [G epoch loss: 2.96110] [Time:0.03329]
	[Generator][Epochs 44/50] [G epoch loss: 3.25575] [Time:0.04258]
	[Generator][Epochs 45/50] [G epoch loss: 3.48814] [Time:0.03927]
	[Generator][Epochs 46/50] [G epoch loss: 3.50928] [Time:0.04340]
	[Generator][Epochs 47/50] [G epoch loss: 3.20343] [Time:0.04439]
	[Generator][Epochs 48/50] [G epoch loss: 3.04184] [Time:0.04331]
	[Generator][Epochs 49/50] [G epoch loss: 2.78207] [Time:0.04216]
	[Generator][Epochs 50/50] [G epoch loss: 2.57355] [Time:0.04566]
[Epoch 3/200]
	[Discriminator][Epochs 1/15] [D epoch loss: 1.36963] [Time:10.95066] 
	[Discriminator][Epochs 2/15] [D epoch loss: 0.39934] [Time:10.86359] 
	[Discriminator][Epochs 3/15] [D epoch loss: 0.16168] [Time:10.61934] 
	[Discriminator][Epochs 4/15] [D epoch loss: 0.08984] [Time:10.96713] 
	[Discriminator][Epochs 5/15] [D epoch loss: 0.06438] [Time:11.09142] 
	[Discriminator][Epochs 6/15] [D epoch loss: 0.05459] [Time:10.96804] 
	[Discriminator][Epochs 7/15] [D

	[Generator][Epochs 35/50] [G epoch loss: 0.07437] [Time:0.04383]
	[Generator][Epochs 36/50] [G epoch loss: 0.06562] [Time:0.04459]
	[Generator][Epochs 37/50] [G epoch loss: 0.07372] [Time:0.04357]
	[Generator][Epochs 38/50] [G epoch loss: 0.07800] [Time:0.03703]
	[Generator][Epochs 39/50] [G epoch loss: 0.08113] [Time:0.03742]
	[Generator][Epochs 40/50] [G epoch loss: 0.08353] [Time:0.03704]
	[Generator][Epochs 41/50] [G epoch loss: 0.07868] [Time:0.03716]
	[Generator][Epochs 42/50] [G epoch loss: 0.07565] [Time:0.03732]
	[Generator][Epochs 43/50] [G epoch loss: 0.08298] [Time:0.03749]
	[Generator][Epochs 44/50] [G epoch loss: 0.07270] [Time:0.03697]
	[Generator][Epochs 45/50] [G epoch loss: 0.07881] [Time:0.03782]
	[Generator][Epochs 46/50] [G epoch loss: 0.07099] [Time:0.03728]
	[Generator][Epochs 47/50] [G epoch loss: 0.07797] [Time:0.03748]
	[Generator][Epochs 48/50] [G epoch loss: 0.08836] [Time:0.03701]
	[Generator][Epochs 49/50] [G epoch loss: 0.07670] [Time:0.03696]
	[Generato

	[Generator][Epochs 27/50] [G epoch loss: 0.09413] [Time:0.04284]
	[Generator][Epochs 28/50] [G epoch loss: 0.11042] [Time:0.04990]
	[Generator][Epochs 29/50] [G epoch loss: 0.12244] [Time:0.04403]
	[Generator][Epochs 30/50] [G epoch loss: 0.12015] [Time:0.04049]
	[Generator][Epochs 31/50] [G epoch loss: 0.12454] [Time:0.04103]
	[Generator][Epochs 32/50] [G epoch loss: 0.15876] [Time:0.03622]
	[Generator][Epochs 33/50] [G epoch loss: 0.17752] [Time:0.03588]
	[Generator][Epochs 34/50] [G epoch loss: 0.16102] [Time:0.05004]
	[Generator][Epochs 35/50] [G epoch loss: 0.14109] [Time:0.05942]
	[Generator][Epochs 36/50] [G epoch loss: 0.13527] [Time:0.04724]
	[Generator][Epochs 37/50] [G epoch loss: 0.16524] [Time:0.04675]
	[Generator][Epochs 38/50] [G epoch loss: 0.17107] [Time:0.05098]
	[Generator][Epochs 39/50] [G epoch loss: 0.22962] [Time:0.04719]
	[Generator][Epochs 40/50] [G epoch loss: 0.27186] [Time:0.03560]
	[Generator][Epochs 41/50] [G epoch loss: 0.22899] [Time:0.04086]
	[Generato

	[Generator][Epochs 19/50] [G epoch loss: -0.25704] [Time:0.03524]
	[Generator][Epochs 20/50] [G epoch loss: -0.27122] [Time:0.05110]
	[Generator][Epochs 21/50] [G epoch loss: -0.28304] [Time:0.04481]
	[Generator][Epochs 22/50] [G epoch loss: -0.28102] [Time:0.04328]
	[Generator][Epochs 23/50] [G epoch loss: -0.22196] [Time:0.04136]
	[Generator][Epochs 24/50] [G epoch loss: -0.21805] [Time:0.04301]
	[Generator][Epochs 25/50] [G epoch loss: -0.19987] [Time:0.04243]
	[Generator][Epochs 26/50] [G epoch loss: -0.21008] [Time:0.05262]
	[Generator][Epochs 27/50] [G epoch loss: -0.19514] [Time:0.05155]
	[Generator][Epochs 28/50] [G epoch loss: -0.16602] [Time:0.03309]
	[Generator][Epochs 29/50] [G epoch loss: -0.15586] [Time:0.03231]
	[Generator][Epochs 30/50] [G epoch loss: -0.17823] [Time:0.03673]
	[Generator][Epochs 31/50] [G epoch loss: -0.14746] [Time:0.04082]
	[Generator][Epochs 32/50] [G epoch loss: -0.13353] [Time:0.03818]
	[Generator][Epochs 33/50] [G epoch loss: -0.09871] [Time:0.04

...

	[Generator][Epochs 36/50] [G epoch loss: -0.10083] [Time:0.05042]
	[Generator][Epochs 37/50] [G epoch loss: -0.11141] [Time:0.05050]
	[Generator][Epochs 38/50] [G epoch loss: -0.11204] [Time:0.04847]
	[Generator][Epochs 39/50] [G epoch loss: -0.07229] [Time:0.05049]
	[Generator][Epochs 40/50] [G epoch loss: -0.08519] [Time:0.05042]
	[Generator][Epochs 41/50] [G epoch loss: -0.08772] [Time:0.05057]
	[Generator][Epochs 42/50] [G epoch loss: -0.07754] [Time:0.05099]
	[Generator][Epochs 43/50] [G epoch loss: -0.08648] [Time:0.05137]
	[Generator][Epochs 44/50] [G epoch loss: -0.12386] [Time:0.05233]
	[Generator][Epochs 45/50] [G epoch loss: -0.17677] [Time:0.04681]
	[Generator][Epochs 46/50] [G epoch loss: -0.12525] [Time:0.05261]
	[Generator][Epochs 47/50] [G epoch loss: -0.09587] [Time:0.05304]
	[Generator][Epochs 48/50] [G epoch loss: -0.09121] [Time:0.05224]
	[Generator][Epochs 49/50] [G epoch loss: -0.05683] [Time:0.04932]
	[Generator][Epochs 50/50] [G epoch loss: -0.04767] [Time:0.05

	[Generator][Epochs 27/50] [G epoch loss: -0.07715] [Time:0.04311]
	[Generator][Epochs 28/50] [G epoch loss: -0.03926] [Time:0.04359]
	[Generator][Epochs 29/50] [G epoch loss: -0.03788] [Time:0.04322]
	[Generator][Epochs 30/50] [G epoch loss: -0.02915] [Time:0.04351]
	[Generator][Epochs 31/50] [G epoch loss: -0.04476] [Time:0.04287]
	[Generator][Epochs 32/50] [G epoch loss: -0.02916] [Time:0.04272]
	[Generator][Epochs 33/50] [G epoch loss: -0.03220] [Time:0.04353]
	[Generator][Epochs 34/50] [G epoch loss: -0.04936] [Time:0.04272]
	[Generator][Epochs 35/50] [G epoch loss: -0.05541] [Time:0.04301]
	[Generator][Epochs 36/50] [G epoch loss: -0.06951] [Time:0.04226]
	[Generator][Epochs 37/50] [G epoch loss: -0.02458] [Time:0.04239]
	[Generator][Epochs 38/50] [G epoch loss: -0.03191] [Time:0.04320]
	[Generator][Epochs 39/50] [G epoch loss: -0.02165] [Time:0.04205]
	[Generator][Epochs 40/50] [G epoch loss: -0.03657] [Time:0.04343]
	[Generator][Epochs 41/50] [G epoch loss: -0.02430] [Time:0.04

	[Generator][Epochs 18/50] [G epoch loss: -0.01209] [Time:0.03358]
	[Generator][Epochs 19/50] [G epoch loss: -0.01447] [Time:0.03395]
	[Generator][Epochs 20/50] [G epoch loss: -0.00965] [Time:0.03359]
	[Generator][Epochs 21/50] [G epoch loss: -0.00273] [Time:0.03348]
	[Generator][Epochs 22/50] [G epoch loss: -0.00489] [Time:0.03855]
	[Generator][Epochs 23/50] [G epoch loss: -0.01225] [Time:0.05049]
	[Generator][Epochs 24/50] [G epoch loss: -0.01806] [Time:0.05078]
	[Generator][Epochs 25/50] [G epoch loss: -0.01029] [Time:0.05056]
	[Generator][Epochs 26/50] [G epoch loss: -0.01844] [Time:0.05087]
	[Generator][Epochs 27/50] [G epoch loss: -0.01572] [Time:0.04016]
	[Generator][Epochs 28/50] [G epoch loss: -0.01572] [Time:0.04258]
	[Generator][Epochs 29/50] [G epoch loss: -0.03226] [Time:0.03862]
	[Generator][Epochs 30/50] [G epoch loss: -0.01297] [Time:0.04289]
	[Generator][Epochs 31/50] [G epoch loss: -0.02745] [Time:0.04263]
	[Generator][Epochs 32/50] [G epoch loss: -0.01657] [Time:0.04

	[Generator][Epochs 9/50] [G epoch loss: -0.07091] [Time:0.04381]
	[Generator][Epochs 10/50] [G epoch loss: -0.06097] [Time:0.04372]
	[Generator][Epochs 11/50] [G epoch loss: -0.06091] [Time:0.04395]
	[Generator][Epochs 12/50] [G epoch loss: -0.06320] [Time:0.04440]
	[Generator][Epochs 13/50] [G epoch loss: -0.07296] [Time:0.04396]
	[Generator][Epochs 14/50] [G epoch loss: -0.05175] [Time:0.04345]
	[Generator][Epochs 15/50] [G epoch loss: -0.05874] [Time:0.04360]
	[Generator][Epochs 16/50] [G epoch loss: -0.05529] [Time:0.04293]
	[Generator][Epochs 17/50] [G epoch loss: -0.05001] [Time:0.04408]
	[Generator][Epochs 18/50] [G epoch loss: -0.02717] [Time:0.04432]
	[Generator][Epochs 19/50] [G epoch loss: -0.03740] [Time:0.04369]
	[Generator][Epochs 20/50] [G epoch loss: -0.04112] [Time:0.04454]
	[Generator][Epochs 21/50] [G epoch loss: -0.02598] [Time:0.04326]
	[Generator][Epochs 22/50] [G epoch loss: -0.03244] [Time:0.04327]
	[Generator][Epochs 23/50] [G epoch loss: -0.04721] [Time:0.043

	[Discriminator][Epochs 15/15] [D epoch loss: 0.03734] [Time:7.95479] 
	[Generator][Epochs 1/50] [G epoch loss: -0.21052] [Time:0.04485]
	[Generator][Epochs 2/50] [G epoch loss: -0.14433] [Time:0.04379]
	[Generator][Epochs 3/50] [G epoch loss: -0.11286] [Time:0.04590]
	[Generator][Epochs 4/50] [G epoch loss: -0.16058] [Time:0.04452]
	[Generator][Epochs 5/50] [G epoch loss: -0.14217] [Time:0.04523]
	[Generator][Epochs 6/50] [G epoch loss: -0.12505] [Time:0.04464]
	[Generator][Epochs 7/50] [G epoch loss: -0.12367] [Time:0.04503]
	[Generator][Epochs 8/50] [G epoch loss: -0.10767] [Time:0.04491]
	[Generator][Epochs 9/50] [G epoch loss: -0.09090] [Time:0.04364]
	[Generator][Epochs 10/50] [G epoch loss: -0.11063] [Time:0.04352]
	[Generator][Epochs 11/50] [G epoch loss: -0.09887] [Time:0.04311]
	[Generator][Epochs 12/50] [G epoch loss: -0.09144] [Time:0.04371]
	[Generator][Epochs 13/50] [G epoch loss: -0.09937] [Time:0.04350]
	[Generator][Epochs 14/50] [G epoch loss: -0.07464] [Time:0.04327]
